### **Importando Bibliotecas Necessárias para o Desenvolvimento**

In [1]:
import os
import heapq
import tempfile

### Função Para Mesclar os Arquivos

In [2]:
def merge_sorted_files(input_files, output_file):
    with open(output_file, 'wb') as output:
        # Abre os arquivos de entrada e cria iteradores para cada um
        iterators = [iter(open(file, 'rb')) for file in input_files]

        # Cria uma heap (fila de prioridade) para armazenar as linhas atuais de cada arquivo
        heap = [(next(iterator), iterator) for iterator in iterators]

        # Converte a heap em uma heap mínima
        heapq.heapify(heap)

        # Mescla os arquivos
        while heap:
            # Pega o menor elemento da heap
            line, iterator = heapq.heappop(heap)

            # Escreve a linha no arquivo de saída
            output.write(line)

            # Obtém a próxima linha do mesmo arquivo
            next_line = next(iterator, None)

            # Se o arquivo não estiver vazio, adicione a próxima linha à heap
            if next_line is not None:
                heapq.heappush(heap, (next_line, iterator))

### Função Principal de Merge Sort Externo

In [4]:
def external_merge_sort(input_file, output_file, chunk_size, max_memory):
    # Divide o arquivo de entrada em pedaços menores
    chunks = []
    with open(input_file, 'rb') as f:
        chunk = []
        current_chunk_size = 0
        for line in f:
            chunk.append(line)
            current_chunk_size += len(line)

            # Se o tamanho atual do chunk exceder o limite, salva o chunk em um arquivo temporário
            if current_chunk_size >= chunk_size:
                temp_file = tempfile.NamedTemporaryFile(delete=False, mode='wb')
                temp_file.write(b''.join(chunk))
                temp_file.close()
                chunks.append(temp_file.name)

                # Reinicia o chunk e o tamanho atual
                chunk = []
                current_chunk_size = 0

    # Ordena cada chunk individualmente
    for chunk_file in chunks:
        lines = open(chunk_file, 'rb').readlines()
        lines.sort()
        with open(chunk_file, 'wb') as f:
            f.writelines(lines)

    # Realiza a intercalação externa dos chunks
    merge_sorted_files(chunks, output_file)

    # Limpa os arquivos temporários
    for chunk_file in chunks:
        os.remove(chunk_file)

### Configurações e Chamada da Função Principal

In [5]:
if __name__ == "__main__":
    # Solicita ao usuário que escolha entre duas opções de limite de memória RAM
    print("Escolha o limite de memória RAM:")
    print("Opção 01 - 5MB")
    print("Opção 02 - 100MB")
    opcao = input("Digite a opção desejada (1 ou 2): ")

    if opcao == "1":
        max_memory = 5 * 1024 * 1024  # 5 MB
    elif opcao == "2":
        max_memory = 100 * 1024 * 1024  # 100 MB
    else:
        print("Opção inválida. Escolha 1 ou 2.")
        exit()

    input_file = "./1m.txt"
    output_file = "./UUIDs_ordenadas.txt"
    chunk_size = max_memory // 2  # Usa metade do limite para os chunks

    # Chama a função principal
    external_merge_sort(input_file, output_file, chunk_size, max_memory)

Escolha o limite de memória RAM:
Opção 01 - 5MB
Opção 02 - 100MB
